In [12]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
"""     ====>     initial parameters     <====     """
n = 24   #number of QDs
p = 12   #number of permanent charges (PCs)
vol = 1   #applied voltage to capacitors in inner disck ("permanent charges"/charged segments)
cd = 0.394    #capacitance density in F/m^2 of inner segment capacitors
e = 1.60217662e-19
R = 3   #radius of QD setup
Rp = 2   #radius of rotating disc
r0 = R - Rp   #distance between roatating disc and centre of QD setup above
thc = 30e-9   #approximate thickeness of inner disc
ro = 3e3 #kg/m^3, desity of inner disc material (approxmiate)
eps = 8.854e-12 #currently set to epsilon 0
k = 1/(4*math.pi*eps)
ts = 30e-9 #double quoted 15ns, time spent stopped between steps, time needed to discharge and charge quantum dot

In [14]:
def intertia( ro, thc, Rp):
    """Calculates inertia of inner rotating disk"""
    I = 0.5 * ro * math.pi * thc * ( Rp ** 4 )
    return I

In [15]:
def permanent_charge_magnitude( Rp, p , cd, vol):
    """Calculates magnitude of total charge stored in inner segment"""
    segment_area = (3*math.pi*(Rp**2))/(4*p) #see notebook for deriviation
    capacitance = cd * segment_area #capacitance of one segment
    stored_charge = capacitance * vol #stored charge in one segment
    return stored_charge

In [16]:
def permanent_charges( Rp, p , cd, vol):
    PCs = {}
    for i in range(p):
        theta = (2*math.pi/p)*i
        xp = Rp*math.cos(theta)
        yp = Rp*math.sin(theta)
        if i%2 == 0:
            p_ch = permanent_charge_magnitude( Rp, p , cd, vol)
        else:
            p_ch = -permanent_charge_magnitude( Rp, p , cd, vol)
        PCs.update({i:[xp,yp,theta, p_ch]})
    return PCs

In [17]:
def quantum_dots( n, e):
    QDs = {}
    for i in range(n):
        fi = (2*math.pi/n)*i
        x = R*math.cos(fi)
        y = R*math.sin(fi)
        if i in range(1, 24, 4):
            q_ch = 2*e
        elif i in range(3,24,4):
            q_ch = -2*e
        else:
            q_ch = 0
        QDs.update({i:[x,y,fi,q_ch]})
    return QDs

In [18]:
def tangential_component(n, R, r0, PCs, QDs):
    F_c = 0; #To work out force in polar I need to know distance from the PC to each QD
    F_t = 0
    ri = math.sqrt(2*R*(R-r0)*(1-math.cos(QDs[i][2]))+r0**2)
    F_c += k*PCs[0][3]*QDs[i][3]/(ri**2)
    d = 2*R*math.sin(QDs[i][2]/2)
    alpha = math.acos(((r0**2)+(ri**2)-(d**2))/(2*r0*ri))
    if i > n/2:
        alpha = -alpha
    F_t += F_c*math.sin(alpha)
    #print F_t
    return F_t

In [19]:
def tangential_component(n, e, R, r0, gamma):
    F_c = 0; #To work out force in polar I need to know distance from the PC to each QD
    F_t = 0
    QDs = quantum_dots(n,e)
    for i in range(n):
        Q = QDs[i]
        ri = math.sqrt(2*R*(R-r0)*(1-math.cos(QDs[i][2]-gamma))+r0**2)
        F_c += k*PCs[0][3]*Q[3]/(ri**2)
        d = 2*R*math.sin((QDs[i][2]-gamma)/2)
        alpha = math.acos(((r0**2)+(ri**2)-(d**2))/(2*r0*ri))
        if i > n/2:
            alpha = -alpha
        F_t += F_c*math.sin(alpha)
    return F_t

In [20]:
def torque (F_t, Rp, p):
    torque = F_t * Rp * p
    return torque

In [21]:
def time_per_increment( phi, Inertia, torque):
    """Time taken to turn angle phi"""
    tau = 0
    tau = math.sqrt( ( 2 * phi * Inertia ) / abs(torque) ) #from rotational equivalent of SUVAT
    return tau

In [22]:
deg15 = math.pi*15/180
PCs = permanent_charges( Rp, p , cd, vol)
for gamma in np.arange(0, deg15 , deg15/30):
    print tangential_component(n, e, R, r0, gamma)

2.73045406764e-10
2.90381142375e-10
3.07554295393e-10
3.24563697968e-10
3.41403659705e-10
3.58063649206e-10
3.74528042812e-10
3.90775943079e-10
4.06781069191e-10
4.22511721184e-10
4.37930819315e-10
4.52996019336e-10
4.67659903639e-10
4.81870247354e-10
4.95570357397e-10
5.08699481268e-10
5.21193281125e-10
5.32984367263e-10
5.44002883775e-10
5.54177137832e-10
5.63434262816e-10
5.71700904504e-10
5.78903918715e-10
5.84971068365e-10
5.89831707725e-10
5.93417441975e-10
5.95662750746e-10
5.96505565404e-10
5.95887791217e-10
5.93755767191e-10


In [23]:
QDs = quantum_dots( n, e)
for i in range(n):
    for j in np.arange(0,30,0.5):
        pos = QDs[i][2] + j